In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

import GCode
import GRBL


# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

"Napkin" scratches.

![](http://luckofthedraw.fun/.imgs/brick_0003.jpeg)

Drawn by hand. ~18mm brick height.

![](http://luckofthedraw.fun/.imgs/brick_0001.jpeg)

> Standard bricks. The standard co-ordinating size for brickwork is 225 mm x 112.5 mm x 75 mm (length x depth x height). This includes 10 mm mortar joints, and so the standard size for a brick itself is 215 mm x 102.5 mm x 65 mm (length x depth x height).

In [2]:
# Standard brick dimensions.
BrickHeight = 65  # [mm]
BrickLength = 225  # [mm]
BrickDepth = 12.5  # [mm]
BrickRatio = 215 / 65  # [dimensionless]


In [3]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 2 * BlockHeight  # mm


In [4]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 5  # [dimensionless]

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows  # [mm]
L_Block_Brick = H_Block_Brick * BrickRatio  # [mm]


# Code:

In [5]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    np.eye(2),  # Do nothing, for debugging.
    flip,  # Flip the matrix, reduces travel time.
)
vertical_brick_lines_tuple = (
    np.arange(L_Block_Brick, BlockLength, L_Block_Brick),# Odd rows.
    np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick), # Even rows.
)
horizontal_brick_lines = np.linspace(0, BlockHeight, N_BrickRows, endpoint=False)

Lines parallel to the X-axis.
Separates rows of bricks.

In [7]:
horizontal_brick_lines

array([ 0. , 17.8, 35.6, 53.4, 71.2])

Lines parallel to the Y-axis.

In [47]:
vertical_brick_lines_tuple


(array([ 58.87692308, 117.75384615, 176.63076923]),
 array([ 29.43846154,  88.31538462, 147.19230769]))

In [49]:
points = list()

lines = list()

for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )

    
    
    

In [46]:
row_line_points

array([[  0. ,  71.2],
       [178. ,  71.2]])

In [15]:
GCode.Line(points=row_line_points)

Line<cut=178.0mm, F=300, S=150>

In [32]:
BlockBrick = GCode.Program()

In [40]:
BlockBrick.lines=list()

for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )
    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]
    
    row_line_points = np.matmul(transform, row_line_points)
    line_ = GCode.Line(
        points=row_line_points,
    )
    
    BlockBrick.lines.append(line_)
    
    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, len(vertical_brick_lines_tuple))]
    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]
    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]
        column_line_points = (
            np.array(
                [
                    [start_point_y, vertical_brick_line],
                    [end_point_y, vertical_brick_line],
                ]
            ),
        )
        column_line_points = np.matmul(transform, column_line_points)
        line_ = GCode.Line(column_line_points)
        BlockBrick.lines.append(line_)

In [41]:
BlockBrick

Program<lines=16>

In [42]:
BlockBrick.lines

[Line<cut=178.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=178.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=178.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=178.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>,
 Line<cut=0.0mm, F=300, S=150>]

In [43]:
BlockBrick.lines[0]

Line<cut=178.0mm, F=300, S=150>

In [52]:
BlockBrick.lines[1]

Line<cut=0.0mm, F=300, S=150>

In [53]:
BlockBrick.lines[1].points

array([[[ 0.        , 29.43846154],
        [17.8       , 29.43846154]]])

In [54]:
BlockBrick.lines[1].points.shape

(1, 2, 2)

In [56]:
BlockBrick.lines[1].points.flatten()

array([ 0.        , 29.43846154, 17.8       , 29.43846154])

In [57]:
help(BlockBrick.lines[1].points.flatten)

Help on built-in function flatten:

flatten(...) method of numpy.ndarray instance
    a.flatten(order='C')
    
    Return a copy of the array collapsed into one dimension.
    
    Parameters
    ----------
    order : {'C', 'F', 'A', 'K'}, optional
        'C' means to flatten in row-major (C-style) order.
        'F' means to flatten in column-major (Fortran-
        style) order. 'A' means to flatten in column-major
        order if `a` is Fortran *contiguous* in memory,
        row-major order otherwise. 'K' means to flatten
        `a` in the order the elements occur in memory.
        The default is 'C'.
    
    Returns
    -------
    y : ndarray
        A copy of the input array, flattened to one dimension.
    
    See Also
    --------
    ravel : Return a flattened array.
    flat : A 1-D flat iterator over the array.
    
    Examples
    --------
    >>> a = np.array([[1,2], [3,4]])
    >>> a.flatten()
    array([1, 2, 3, 4])
    >>> a.flatten('F')
    array([1, 3, 2, 4]

In [59]:
for order in ('C', 'F', 'A', 'K'):
    print(BlockBrick.lines[1].points.flatten(order=order))

[ 0.         29.43846154 17.8        29.43846154]
[ 0.         17.8        29.43846154 29.43846154]
[ 0.         29.43846154 17.8        29.43846154]
[ 0.         29.43846154 17.8        29.43846154]


In [60]:
BlockBrick.lines[1].points.squeeze()

array([[ 0.        , 29.43846154],
       [17.8       , 29.43846154]])

In [ ]:
BlockBrick.lines[1]